# AI-FW2 効果測定1 問題3

**環境設定**  
google colaboratoryでMecabを使うため、以下のセルを実行してください  
(数分かかることがありますが、3分経過しても実行が終わらない場合はランタイムを出荷状態に)

---
ここから

In [1]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [2]:
# 念のためパスの確認
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd


ここまで

---

**自動作文プログラム**  
ここから先がプログラムです。  
空欄個所を実装してください。

In [3]:
import MeCab
import os,json,random

テキストファイルを読み込み、マルコフ辞書への登録処理を呼び出す  
※空欄あり

In [4]:
# ファイルを読み込み、マルコフ辞書に登録する関数を作成する
def make_markof(filename):
    # withを使用してテキストファイルを開く
    with open(filename) as f:
        # 1行ずつ読み込む
        line = f.readline()

        # テキストファイルの最終行までループする
        while line:
            text = line.rstrip()

            # 文章整形
            if text[-1] != "。" and text[-1] != "?" and text[-1] != "？" :
                text +="。"

            # 形態素解析
            tagger = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
            tagger.parse('')
            node =  tagger.parseToNode(text)

            # 形態素解析の結果から、単語と品詞情報を抽出
            wordlist = []
            while node is not None:
                hinshi = node.feature.split(',')[0]
                if  hinshi not  in ['BOS/EOS']:
                    wordlist.append([node.surface,hinshi])
                node = node.next
            
            # マルコフ連鎖の辞書に登録
            regist_dic(wordlist)

            line = f.readline()

マルコフ辞書への登録処理  
※空欄あり

In [5]:
dict_file = "markov_dict.json"
dic = {}

# 名詞をカウントするための辞書型の変数を作成する
noun_dic = {}

# 辞書への登録
def regist_dic(wordlist):
    global dic
    w1 = ''
    w2 = ''
    
    # 要素が3未満の場合は、何もしない
    if len(wordlist) < 3 :
      return

    for w in wordlist :
        word = w[0]

        # 辞書に登録する単語が名詞の場合はカウントする
        if w[1] == '名詞':
            if w[0] in noun_dic:
                noun_dic[w[0]] += 1
            else:
                noun_dic[w[0]] = 1

        if word == '' or  word == '\r\n' or word == '\n' :
            continue

        # 辞書に単語を設定
        if w1 and w2 :
            set_dic(dic,w1, w2, word)

        # 文末を表す語の場合、連鎖をクリアする
        if word == '。' or word == '?' or  word == '？' :
            w1 = ''
            w2 = ''
            continue
        # 次の前後関係を登録するために、単語をスライド
        w1, w2 = w2, word
    
    # 辞書を保存
    json.dump(dic, open(dict_file, 'w', encoding='utf-8'))

# 辞書に単語を設定
def set_dic(dic, w1, w2, w3):
    # 新しい単語の場合は、新しい辞書オブジェクトを作成
    # ・dicのキーにw1が存在しないときは、キーを追加し、空の辞書をセットする
    if w1 not in dic : dic[w1] = {}

    # ・dic[w1]のキーにw2が存在しないときは、キーを追加し、空の辞書をセットする
    if w2 not in dic[w1] : dic[w1][w2] = {}

    # ・dic[w1][w2]のキーにw3が存在しないときは、キーを追加し、0をセットする
    if w3 not in dic[w1][w2] : dic[w1][w2][w3] = 0

    # 単語の出現数をインクリメントする
    dic[w1][w2][w3] += 1

入力に対して自動作文する処理（変更不要）

In [6]:
# 応答文の作成
def make_response(word):
    res = []
    
    # 「名詞」/「形容詞」/「動詞」は、文章の意図を示していることが多いと想定し、始点の単語とする。
    w1 = word
    res.append(w1)
    w2 = word_choice(dic[w1])
    res.append(w2)
    while True:
        # w1,w2の組み合わせから予想される、単語を選択
        if w1 in dic and w2 in dic[w1] : w3 = word_choice(dic[w1][w2])
        else : w3 = ""
        res.append(w3)
        # 文末を表す語の場合、作文を終了
        if w3 == "。" or w3 == "?" or  w3 == "？"  or w3 == "" :  break
        # 次の単語を選択するために、単語をスライド
        w1, w2 = w2, w3
    return "".join(res)
        
def word_choice(candidate):
    keys = candidate.keys()
    return random.choice(list(keys))

# メイン処理
def play_chat():

  # 辞書がすでに存在する場合は、最初に読み込む
  if os.path.exists(dict_file):
          dic = json.load(open(dict_file,"r"))
          
  while True:
      # 標準入力から入力を受け付け、「さようなら」が入力されるまで続ける
      text = input("You -> ")
      if text == "" or text == "さようなら" : 
          print("Bot -> さようなら")
          break

      # 文章整形
      if text[-1] != "。" and text[-1] != "?" and text[-1] != "？" : text +="。"
      
      # 形態素解析
      tagger = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")
      tagger.parse("")
      node =  tagger.parseToNode(text)
      
      # 形態素解析の結果から、単語と品詞情報を抽出
      wordlist = []
      while node is not None:
          hinshi = node.feature.split(",")[0]
          if  hinshi not  in ["BOS/EOS"]:
              wordlist.append([node.surface,hinshi])
          node = node.next

      # 応答文の作成
      for w in wordlist:
          word = w[0]
          hinshi = w[1]
          # 品詞が「感動詞」の場合は、単語をそのまま返す
          if hinshi in [ "感動詞"] : 
              print("Bot -> " + word)
              break
          # 品詞が「名詞」「形容詞」「動詞」の場合で、かつ、辞書に単語が存在する場合は、作文して返す
          elif (hinshi in [ "名詞" ,"形容詞","動詞"]) and (word in dic):
              print("Bot -> " + make_response(word))
              break

マルコフ辞書を作成する  
(数分かかることがあります)

In [7]:
# make_markof関数を使用して、テキストファイル「kouka_1_3_data.txt」のマルコフ辞書を作成する
make_markof('kouka_1_3_data.txt')

# マルコフ辞書に登録された名詞と出現回数を表示する(noun_dicの内容)
print(noun_dic)

{'吾輩は猫である': 3, '名前': 15, 'どこ': 38, 'かとん': 1, '見当': 8, '何': 229, 'した': 5, '所': 44, 'いた事': 3, '記憶': 12, '吾輩': 250, 'ここ': 27, '人間': 95, 'もの': 318, 'あと': 24, 'それ': 162, '書生': 17, '中': 84, '一番': 11, '獰悪': 1, '種族': 2, 'そうだ': 22, 'の': 525, '我々': 6, '話': 21, '当時': 3, '考': 9, '彼': 74, '掌': 3, 'スー': 2, '時': 143, '感じ': 13, '上': 117, '顔': 116, '始': 1, '妙': 42, '今': 61, '一': 100, '毛': 10, '装飾': 2, 'はず': 21, '薬缶': 2, 'その後': 9, '猫': 138, '輪': 5, '度': 41, 'た事': 3, '真中': 13, '突起': 1, '穴': 10, '煙': 5, '咽': 3, 'これ': 120, '煙草': 8, '事': 447, '頃': 21, '裏': 19, '心持': 8, '非常': 16, '速力': 1, '運転': 1, '自分': 67, '眼': 69, '胸': 7, '音': 26, '火': 7, '気': 91, 'たくさん': 3, '兄弟': 1, '疋': 5, '肝心': 4, '母親': 1, '姿': 5, '容子': 6, '藁': 1, '急': 35, '笹原': 2, 'ようやく': 13, '思い': 3, '向う': 12, '池': 9, '前': 55, '分別': 2, '迎': 2, 'ニャー': 2, '試み': 1, '誰': 40, '池の上': 1, '風': 13, '日': 47, '暮れ': 2, '腹': 15, '声が出ない': 2, '仕方がない': 22, '食物': 1, '決心': 9, '左': 13, 'そこ': 15, '我慢': 8, '無理やり': 1, '竹垣': 2, '邸': 10, '内': 27, '縁': 3, '不思議': 26, '路傍': 1, 

▶実行結果（抜粋）  
{'吾輩は猫である': 3, '名前': 15, 'どこ': 38, 'かとん': 1, '見当': 8, ～(中略)～ , 'オタンチン・パレオロガス': 4, 'すそ': 1}

自動作文を実行する

In [8]:
# play_chat関数を呼び出し、Botに3つ以上の文章を作成させる
play_chat()

You -> あなたの名前はなんですか
Bot -> あなた、私しゃこの年に一度にとる。
You -> どうぞ自由に文章を作成してください
Bot -> 自由にならんと主張する。
You -> すみません、分かりませんでした。
Bot -> すみません
You -> さようなら
Bot -> さようなら


▶実行結果（サンプル）  
You -> あなたの名前はなんですか  
Bot -> あなたを可愛がっている小児ですらこの傾向があるさ君は幸い時間も狭められた通りを述べると、残ってる三冊をとったって――一てえ人間ほど不人情なものじゃありませんやね、せっかく御求め被下候。  
You -> どうぞ自由に文章を作成してください  
Bot -> 自由に上下し得ると思ったよいやに瘠せてるじゃありませんわ。  
You -> すみません、分かりませんでした。  
Bot -> すみません  
You -> さようなら  
Bot -> さようなら  